# Project Design Outline (MS1)

Team39: Philipp Arens, Ben Ray, Andy Zhuo

## Introduction

Derivatives play an essential role in many areas of science and engineering such as fluid dynamics, solving ODEs, optimization and - relatedly - machine learning. In general, there exist four main ways of computing derivatives in practice, that is: manual computation (i.e. computing and implementing derivatives by hand), symbolic differentiation (i.e. WolframAlpha or SymPy), numerical differentiation (i.e. finite difference method) and automatic differentiation (AD). Depending on the complexity of the target function, manual differentiation can require significant amounts of time and is often challenging to "debug". Symbolic differentiation alleviates this at the expense of potentially bloated/unitutitive expressions. If analytic derivative expressions are not needed, numerical differentiation, that is approximating derivatives through finite differences, could be considered. This, however, can lead to approximation/floating point errors if the step size is chosen inadequately. 

AD has emerged as a promising way of adressing many of these issues. Though not providing closed form/symbolic expressions, it allows us to compute derivatives with machine precision without introducing large computational overhead. Sparked by recent advances in machine learning, in particular deep neural networks, which use a subclass of AD (i.e. the backpropagation algorithm) in their training phase, AD has shown its potential across a variety of different aplications.

In this project we aim at developing a software package implementing a subclass of AD, namely forward mode AD. We aim at making this software intuitive to use, following best practices in terms of relying on the python data model and style guides.

## Background

Central to automatic differentiation (AD) is use of the chain rule to decompose derivatives into elemtary binary operations (+, -, *, /, **) and unary operations (sin, cos, exp, log, etc.) Using the chain rule allows for two modes of evaluating a derivative:

1. Forward mode AD - the derivative is computed "bottom up" (i.e. fixing the independent variable(s), take the derivative of the innermost function first, and then move up taking derivatives of each sub-expression recursively)
2. Reverse mode AD - the derivative is computed "top down" (i.e. you fix the dependent variable, take the derivative of the outermost function first with respect to its sub-expressions, and then move inwards taking successive derivatives with respect to their inner sub-expressions).

AD can be generalized to both oridinary and partial derivatives using matrix products of Jacobians.

Finally, in forward mode AD, we commonly redefine our arithmetic to use dual numbers of the form $x + x'\epsilon$ where $\epsilon$ is a mathematical object not contained in the set of real numbers, with the property $\epsilon^2 = 0$. This allows us to store the decomposed derivative as a computational graph represented by dual numbers in each node. Despite this abstract implementation, we can still preserve the critical concept of "duck typing": as long as our dual numbers act like an input to their function (e.g. they can be added, multiplied, etc.) then the function will work.

## How to use `ad39`

Users will be able to install our package from the Test PyPi server using pip. Users will also have to `pip install numpy` (and maybe `pip install matplotlib` depending on our extension) - the packages that `ad39` is dependent on. Having installed the package, users will be able to call `import ad39` before using the package to its full effect.

The following provides a summary of how users will be able to perform automatic differentiation with the `ad39` package:

- Instantiate all input variables to your function as nodes:
e.g. given $f(x_1, x_2) = x_1 + x_2$, instantiate `x1 = Node()` and `x2 = Node()`
- Write the function you wish to differentiate in terms of its node inputs
e.g. `def f(x1, x2):
        return x1 + x2`
- Instantiate an object for performing AD:
e.g. `AD = AD(f)`
- Call the forward mode method on the AD object, passing the input values (and, optionally if not one-hot vectors, the seed vector) at which you wish to evaluate the function and its derivative:
e.g. calling `AD.forward_mode(x1=1, x2=1, seed=None)` will return a tuple of $f(1, 1)$ and $f'(1, 1)$

In summary, using the `ad39` package to perform forward mode will look something like the following:

In [ ]:
# Install package
python3 -m pip install --index-url https://test.pypi.org/simple/ --extra-index-url https://pypi.org/ ad39

# Import package
import ad39

# Instantiate nodes
x1 = Node()
x2 = Node()

# Write function you wish to differentiate in terms of its node inputs
def f(x1, x2):
    return x1 + x2

# Instantiate an object for performing AD
AD = AD(f)

# Perform forward mode AD
result = AD.forward_mode(x1=1, x2=1, seed=None) # returns a tuple of $f(1, 1)$ and $f'(1, 1)$

## Software Organization

### Structure

    project
        ├── LICENSE
        ├── README.md
        ├── docs
        ├── project.toml
        ├── setup.cfg
        ├── src
        │   └── ad39_package
        │       ├── __init__.py
        │       ├── core.py
        │       └── helpers.py   
        └── tests
            ├── run_tests.sh
            ├── test_core.py
            └── test_helpers.py

### Modules

Most of the file tree is self explanatory, but to briefly explain the contents of the `ad39_package`:
- `__init__.py` will initialize the entire package.
- `core.py` contains the main classes for AD like the `Node` and `ForwardMode` classes (defined below)
- `helpers.py` contains potential helper functions or classes as (e.g. a `DualNumbers` class)

We may choose to include additional modules as the project grows.

### Tests

Our test suite will live in a designated `test` directory on the root level of the main `project` directory. We envision this to include a test of the main features as well as a test of the additional features of our AD package.

### Package Distribution

We plan to distribute our package using PyPI (i.e PEP517/518). To this end we would use the PyPI Test server and resolve extra dependencies from the main server with the `--extra-index-url` flag.

## Implementation

### Core Data Structures

Our implementation of AD relies on a directed computational graph represented as a set of instantiated Node objects. Each instance of the Node class will store, upon initialization, 
1. `op` - an elementary operation to be performed on the intermediate variable(s) passed into the node.
2. `deriv` - the derivative expression corresponding to `op`, constructed via a helper function.
3. `parent` - an array of at most 2 previous nodes, containing the intermediate variable(s) passed as arguments to this node's `op`.
4. `val` - the function's value evaluated at this step in the computation, initialized as `None` until an input is provided in a call to `ForwardMode`.
5. `derival` - the function's derivative evaluated at this step in the computation, initialized as `None` until an input is provided in a call to `ForwardMode`.

The use of a digraph data structure will allow us to quickly traverse through each intermediate operation when executing forward mode, and the storage of both a function and its derivative in each corresponding node enables the chain rule to be more easily applied upon the evaluation of the elementary operations. While a computational graph is not entirely necessary for the implementation of forward mode, as we have the naive alternative of using dual numbers, constructing a graph will provide great utility in later stages, especially for extensions such as implementing reverse mode or visualizing the computational graph.

### Classes and Attributes

In [ ]:
class Node:

    def __init__(self, **kwargs):
        self.op = kwargs['op']
        self.deriv = grad(self.op, self.args)
        self.parent = kwargs['parent']
        self.val = None
        self.derival = None

    # For any unary operator (e.g. trig, negation, sqrt, etc.)
    # Replace `unop` with name of operator
    def unop(self):
        next_node = Node('unop', [self])
        return next_node
        self.next = next_node

    # For any binary dunder operator (e.g. add, minus, divide, multiply, etc.)
    # Replace `binop` with name of operator and `name` with new operator name
    def __binop__(self, other):
        next_node = Node(name, op = 'binop', parent = [self, other])
        return next_node
        self.next = next_node

class AD:
    """ AD class contains core functionality for permorming forward mode AD, calculating the gradient, and extension functionality like (maybe) drawing the computational graph """

    def __init__(self, f):
        self.nodes = # self.nodes will be a list of nodes or Tree of nodes
        pass

    # Computes forward mode AD
    def forward_mode(self, seed = None, **kwargs):
        pass
    
    # Constructs the gradient of the intermediate function given the provided arguments
    def grad(self, node):
        pass

    # For example
    def draw(self):
        pass

### Operator Overloading

*Graph Implementation*

For the graph implementation of forward mode, and to allow for a potential reverse mode extension, each node in the graph represents an intermediate variable, defined as an output of an operator `op` executed on a set of arguments `parent` from the parent node(s). In order to construct the computational graph for all nodes, we will overload operators such that when the user inputs the function, each operator present in the function will instantiate a new Node storing the operation, and the sub-expression(s) stored as argument(s). The overload template, displayed in the previous section, is different for unary operators, which will be defined in the `Node` class (e.g. def sin(self)), and for binary operators, which will be redefined dunder methods in the Node class (e.g. def \_\_add\_\_(self, other)).

*Dual Numbers*

At this stage, we are still considering instantiating a separate means for performing forward mode AD using dual numbers, and reserving the `Node` class for an extension in which we display the computational graph. If our choice of extension resulted in us using dual numbers (as well as nodes), we would overload the operators in a `DualNumber` class. As the chain rule holds for dual numbers we know that for $f(z) = f(a + b \epsilon) = f(a) + f'(a)b \epsilon$. For example if we wanted to implement $sin(z)$ where $z$ is a dual number we would have to overload the sin operator to return a new `DualNumber` object with the real component $sin(a)$ and the dual component $cos(a) b$. This will need to be implemented for every operator we plan to support in our library.

### External Libraries

- NumPy
- Matplotlib.pyplot (if we include graph functionality)

## License

We decided to go with an MIT (i.e. copyright) license as it permits reuse within proprietary software, provided that all copies of the software or its substantial portions include a copy of the terms of the MIT License and also our original copyright notice. Since we do not hope to commercialize this project, we are happy for others to use it. However, we feel that including the original copyright notice in all derived projects is important, so that future developers understand that the `ad39` package was designed as an educational project (and not more than that), i.e. we want to set developers' expectations for using the `ad39` package by telling them the provenance of the package.

## Feedback

### Milestone 1
- Background
    - We explained the difference between forward and reverse node as computing the derivative "bottom up" and "top down" rather than "from the inside out" and "from the outside in", given the intuition that expressions can be thought of as syntax trees.
    - We clarified that $\epsilon$ is a mathematical object not contained in the set of real numbers, to clarify what we had previously described as an "abstract number".

- How To Use
    - Summarized usage in a code snippet.
    - Changed how you initialized variables as nodes, removing the need for providing a name to the node (which could conflict across variables): i.e. before you initialized as follows `x1 = Node("x1")`, now it is just `x1 = Node()`.

- Software Organization
    - Added a note that we may need more modules in `ad39_package` as the project grows.

- Implementation
    - Removed `node.child` from our implementation of the `Node` class following feedback that "as long as each node keeps reference of its parent(s), you can access the whole graph with the output nodes as a handle." Also removed `node.name` as per earlier feedback - this is no longer necessary, and if we ever need to give a unique ID to nodes, it will be done entirely on the backend.
    - Removed the unintuitive `ForwardMode` class, and instead defined an `AD` class, which now includes: (1) a new `forward_mode` method, (2) the `grad` method (moved from the `Node` class to here, but no longer a static method), and (3) any other methods that will be included in our extension, e.g. `draw` for drawing the computational graph. Note, that the `AD` class object is initialized with the function on which AD will be performed, and we may use a further `Tree` class (or just a list of `Node` objects) under the hood to store the nodes created during the computation.

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=91322482-c0b3-4cf4-b2f2-3cb2acdb509e' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>